# <h1><center> Homework 2 Assignment </center></h1>
### <h1><center> Computer Music Representations and Models Course </center></h1>
#### <h1><center> Acoustics Engineering @ Politecnico di Milano </center></h1>
